## Absolute, Relative, Normalized, Canonical Path 

### Using `os.path`

Current File:
```python
print(__file__)
```

Absolute file path : 
```python
print(os.path.abspath(__file__))
```

File directory name:
```python
print(os.path.dirname(__file__))
```


Pointing to the one level up, relative paths
```python
root = os.path.join(os.path.dirname(__file__), "..")
print(root)
```

Pointing to the one level up, normalized paths
```python
print(os.path.normpath(root))
```

Returns the current working directory, python execution path 


```python
print(os.getcwd())
```

### Using `pathlib` (modern and cleaner)

Some developers prefer `pathlib` (standard since Python 3.4+):

```python
from pathlib import Path

# Script's directory
current_dir = Path(__file__).resolve().parent

# Go to project root and access config
config_path = current_dir.parent / 'config' / 'config.yaml'
```

Canonical

```python
print("Canonical:", current_dir.resolve(strict=True))  # also checks that it exists
```


---


##  Loading resources relative to your project

Given this layout:

```
.
├── config/
│   └── config.yaml
├── main.py
└── utils/
    ├── __init__.py
    └── file_utils.py
```

### 1) The core idea

Inside **`utils/file_utils.py`**, `__file__` *is* defined (because it’s a module file on disk). You can compute paths relative to it. Prefer `pathlib` over `os.path`:

```python
# utils/file_utils.py
from pathlib import Path

def project_root() -> Path:
    # utils/ -> project root (go up 1)
    return Path(__file__).resolve().parent.parent

def resource_path(*parts: str) -> Path:
    """
    Build a path under the project root in an OS-safe way.
    Example: resource_path("config", "config.yaml")
    """
    return project_root().joinpath(*parts)
```

Usage from anywhere in your code:

```python
from utils.file_utils import resource_path
cfg = resource_path("config", "config.yaml")
print(cfg)                 # e.g., /home/user/myproj/config/config.yaml
text = cfg.read_text()     # Path objects have handy IO helpers
```

#### Why this is better than `os.path`:

* `Path(__file__).resolve()` normalizes `..` and resolves symlinks.
* Methods like `.parent`, `.joinpath`, `.read_text()` are clear and cross-platform.
* Returns `Path` instead of `str`, which is nicer to work with.

---

### 2) Common pitfalls & how to avoid them

### a) Running from a different CWD

Never rely on `os.getcwd()` for locating project files. Using `__file__` (as above) avoids surprises when `main.py` is launched from another directory.

### b) Jupyter notebooks

Notebooks don’t define `__file__`. If you need the same helper inside a notebook, either:

* Import a module (like `utils/file_utils.py`) that *does* have `__file__`, or
* Detect the environment:

```python
# utils/file_utils.py
from pathlib import Path
import os

def project_root_fallback() -> Path:
    try:
        return Path(__file__).resolve().parent.parent
    except NameError:
        # Likely in a notebook; fallback to CWD
        return Path(os.getcwd()).resolve()
```

(Prefer importing `resource_path` in your notebook rather than redefining it there.)

### c) Packaging & editable installs

If you later package and install your project (`pip install -e .`), files like `config.yaml` should be accessed via **importlib.resources** (so they’re available inside wheels/zip apps). Example:

```python
# If config is inside a Python package, e.g., mypkg/config/config.yaml
from importlib.resources import files

def package_config_path() -> Path:
    return files("mypkg.config").joinpath("config.yaml")
```

If `config/` is *not* a Python package (no `__init__.py`), keep your current approach or make `config` a package.

### d) Symlinks

`Path(__file__).resolve()` gives you the real path (following symlinks). If you *don’t* want symlink resolution (rare), use `Path(__file__).absolute()`.

---

## 3) A tiny loader example

```python
# utils/config_loader.py
from pathlib import Path
import yaml
from .file_utils import resource_path

def load_config(name: str = "config.yaml") -> dict:
    cfg_path: Path = resource_path("config", name)
    if not cfg_path.exists():
        raise FileNotFoundError(f"Config not found: {cfg_path}")
    with cfg_path.open("r", encoding="utf-8") as f:
        return yaml.safe_load(f)
```

Use it in `main.py`:

```python
# main.py
from utils.config_loader import load_config

def main():
    cfg = load_config()
    print(cfg)

if __name__ == "__main__":
    main()
```

---

## 4) Tests (pytest)

```python
# tests/test_paths.py
from utils.file_utils import project_root, resource_path

def test_project_root_exists():
    assert project_root().exists()

def test_config_path_points_to_file():
    assert resource_path("config", "config.yaml").is_file()
```

Run: `pytest -q`

---

## 5) Your original function, improved

Your version:

```python
import os

def get_project_root() -> str:
    root = os.path.join(os.path.dirname(__file__), "..")
    root_norm = os.path.normpath(root)
    return root_norm
```

Modernized and typed:

```python
from pathlib import Path

def project_root() -> Path:
    return Path(__file__).resolve().parent.parent
```

If you really need a string:

```python
str(project_root())
```

---



## 6) When to switch to `importlib.resources`

* If `config.yaml` must be **distributed** with your package and be accessible after `pip install`, place it under a package (e.g., `mypkg/config/config.yaml`) and access via `importlib.resources`:

```python
from importlib.resources import files
import yaml

def load_pkg_config() -> dict:
    with files("mypkg.config").joinpath("config.yaml").open("r", encoding="utf-8") as f:
        return yaml.safe_load(f)
```

This survives wheels and zipapps.

---

